## Beyond Linearity 1

We need to add some additional packages to make this work.

So start with *pip3 install ISLP*.  ISLP is a package from a book entitled 'Introduction to Statistical Learning with Python'.

We'll also need the *pygam* package.

Then *pip3 install pygam*

In [ ]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
import matplotlib.pyplot as plt

from ISLP import load_data
from ISLP.models import (summarize,
                         poly,
                         ModelSpec as MS)
from statsmodels.stats.anova import anova_lm

In [ ]:
from pygam import (s as s_gam,
                   l as l_gam,
                   f as f_gam,
                   LinearGAM,
                   LogisticGAM)

from ISLP.transforms import (BSpline,
                             NaturalSpline)
from ISLP.models import bs, ns
from ISLP.pygam import (approx_lam,
                        degrees_of_freedom,
                        plot as plot_gam,
                        anova as anova_gam)

We are going to start with some data on wages.  We'll look at the data on wages with age as a predictor.

In [ ]:
Wage = load_data('Wage')
print(Wage.info())
y = Wage['wage']
age = Wage['age']
Wage.head()

In [ ]:
plt.scatter( Wage['age'],Wage['wage'],s=1)

# Add labels and title
plt.xlabel('Age')
plt.ylabel('Wage' )
plt.title('Plot of Age vs Wage')
plt.show()

So that scatterplot suggests that we have a non-linear relationship here.

So let's build some non-linear models.

### Polynomial Regression

Polynomial regression means that we have a regression with predictors $x$, $x^2$, $x^3$, ..., to $x^d$ where
$d$ is the degree of the polynomial.  So a cubic is of degree 3 and a linear function would be of degree 1.

We will do this in a special way that make the predictors not be correlated.  Math.

In [ ]:
# This builds the fourth degree polynomial predictors for age
# so that we have in poly_age: age, age^2, age^3, age^4
poly_age = MS([poly('age', degree=4)]).fit(Wage)
# Use the polynomials of age to fit a regression model an 
# ordinary least squares (OLS) one 
M = sm.OLS(y, poly_age.transform(Wage)).fit()
# get the summary for this model
summarize(M)


In the output above, we have the y-intercept (111.7), then the coefficient for age (447.1), the coefficient for age^2 (-478.3),
    the coefficient for age^3 (125.5) and the coefficient for age^4 (-77.9).

For making predictions, we create a set of values in *age_grid* and then we'll match the structure of the polynomials
by transforming them to make them not correlated as we did above.

In [ ]:
#make a set of 100 values from the smallest age to the largest age
age_grid = np.linspace(age.min(),
                       age.max(),
                       100)
print(age_grid)

# put those values into a dataframe called age_df
age_df = pd.DataFrame({'age': age_grid})
# make the values at which we are going to predict and transform them 
Xnew = poly_age.transform(age_df)
# generated predicted values at the values in age_grid
preds = M.get_prediction(Xnew)

#print(preds.summary_frame())
age_df['preds']=preds.predicted_mean


Below we make a plot and add the prediction equation in red to the plot.

In [ ]:
plt.scatter( Wage['age'],Wage['wage'],s=1)

# Add labels and title
plt.xlabel('Age')
plt.ylabel('Wage' )
plt.title('Plot of Age vs Wage')


# Add regression line to plot
plt.plot(age_df['age'],age_df['preds'] , color='red')

plt.show()

The above plot is the fourth order polynomial model for prediction of an individuals wage by age.

Note that the model is not quadratic as it has an extended area of peak which is not 
the form that a quadratic would take.

### Splines

Polynomials give us some flexibility in the relationship between our response/target and our 
features/predictors.  A *spline* is another way to build some flexibility into that relationship.

The idea of a spliced lines, or splines, is to fit a functional curve, by default cubic, to data in a window, then
fit another  curve to the next window, etc through all the windows, and have the lines be spliced together.
Where the windows come together are called knots because they 'tie together' one curve to the next.

We do this by creating bases, see Linear Algebra, as predictors which are effectively perpendicular to each other.

The `BSpline()`  function generates the
entire matrix of basis functions for splines with the specified set of
knots. By default, the B-splines produced are cubic. To change the degree, use
the argument `degree`.

In [ ]:
# BSplines are basis splines (bs)
# basically these are orthogonal, read perpendicular, predictors that ensure there 
# is not much multicollinearity between them
# the knots are the locations on the x axis
bs_1 = BSpline(internal_knots=[25,40,60], intercept=True).fit(age)
# transform the splines to fit the age and the function below
bs_age = bs_1.transform(age)
# get the number of rows and columns for bs_age
bs_age.shape

The next set of code makes the necessary predictors to have splines between knots 
at ages 25, 40 and 60.  

In [ ]:
bs_age = MS([bs('age', degree=3,internal_knots=[25,40,60])])
# MS is a function that takes python sequences and makes them into a matrix
Xbs = bs_age.fit_transform(Wage)
# The fit_transform() method in Python, particularly within the scikit-learn library, 
# serves as a combined operation of fitting and transforming data. 
# It is commonly used in data preprocessing steps for machine learning. 
# The fit() part calculates necessary parameters from the data, 
# such as mean and standard deviation in the case of scaling. 
# The transform() part then applies these calculated parameters to transform the data. 
# Using fit_transform() performs both actions sequentially in a single step, 
# which can be more efficient than calling fit() and transform() separately.

# fit a regression model to these data with y as the response and Xbs as our predictors
# call that model M2
M2 = sm.OLS(y, Xbs).fit()
# get the coefficient summary for M2
summarize(M2)

Notice that there are 6 spline coefficients rather than 7. This is because, by default, `bs()` assumes `intercept=False`, since we typically have an overall intercept in the model. So it generates the spline basis with the given knots,  and then discards one of the basis functions to account for the intercept. 

We could also use the `df` (degrees of freedom) option to specify the complexity of the spline.  We see above that with 3 knots, the spline basis has 6 columns or degrees of freedom.  When we specify `df=6` rather than the actual knots, `bs()` will produce a spline with 3 knots chosen at uniform quantiles of the training data.  

We can see these chosen knots most easily using `Bspline()` directly:

In [ ]:
BSpline(df=6).fit(age).internal_knots_


Next we will make predictions so we can plot and compare how the splines do compared to the polynomial regression.

In [ ]:
# create the data for making predictions based upon the bs_age transformation
Xnew = bs_age.transform(age_df)
# get the predicted values from M2 which is our regression model
preds2 = M2.get_prediction(Xnew)

# M2.get_prediction gives a bunch of output we just wanted the predicted values/means
age_df['preds2']=preds2.predicted_mean

Next make the plots and compare the polynomial in *red* to the splines in *blue*.

In [ ]:
plt.scatter( Wage['age'],Wage['wage'],s=1, color="grey")

# Add labels and title
plt.xlabel('Age')
plt.ylabel('Wage' )
plt.title('Plot of Age vs Wage')


# Add prediction line to plot
plt.plot(age_df['age'],age_df['preds2'] , color='blue')
# Add prediction line to plot
plt.plot(age_df['age'],age_df['preds'],color="red")

plt.show()

The predictions are similar but with some slight differences between the polynomial and the spline fit.
If we were to change the polynomial order or the type of spline we would get different predictions.



Below we will change the type of spline to be linear (degree=1) and refit the model. 

In [ ]:
bs_age2 = MS([bs('age', degree=1,internal_knots=[25,40,60])])
# MS is a function that takes python sequences and makes them into a matrix
Xbs2 = bs_age2.fit_transform(Wage)
# The fit_transform() method in Python, particularly within the scikit-learn library, 
# serves as a combined operation of fitting and transforming data. 
# It is commonly used in data preprocessing steps for machine learning. 
# The fit() part calculates necessary parameters from the data, 
# such as mean and standard deviation in the case of scaling. 
# The transform() part then applies these calculated parameters to transform the data. 
# Using fit_transform() performs both actions sequentially in a single step, 
# which can be more efficient than calling fit() and transform() separately.

# fit a regression model to these data with y as the response and Xbs as our predictors
# call that model M2
M3 = sm.OLS(y, Xbs2).fit()
# get the coefficient summary for M2
summarize(M3)

We've not interpreted these coefficients since they are a challenge to interpret since they depend upon the basis functions that were defined.  What we can tell is that since all of the above coefficients have small p-values, that they are worth keeping in the model.

Next we will get the predictions for the model *M3*.

In [ ]:
Xnew = bs_age2.transform(age_df)

preds3 = M3.get_prediction(Xnew)

#print(preds.summary_frame())
age_df['preds3']=preds3.predicted_mean

In [ ]:
plt.scatter( Wage['age'],Wage['wage'],s=1)

# Add labels and title
plt.xlabel('Age')
plt.ylabel('Wage' )
plt.title('Plot of Age vs Wage')


# Add prediction line to plot
plt.plot(age_df['age'],age_df['preds2'] , color='blue')
# Add prediction line to plot
plt.plot(age_df['age'],age_df['preds'],color="red")
# Add prediction line for degree =2 spline
plt.plot(age_df['age'],age_df['preds3'],color="black")

plt.show()

Let's change the y-axis so that we can more closely see that is going on.  

We'll do that below with the *plt.ylim* command.

In [ ]:
# note here that the addition of s=1 makes for 
plt.scatter( Wage['age'],Wage['wage'],s=1)

# Add labels and title
plt.xlabel('Age')
plt.ylabel('Wage' )
plt.title('Plot of Age vs Wage')


# Add prediction line to plot
plt.plot(age_df['age'],age_df['preds2'] , color='green')
# Add prediction line to plot
plt.plot(age_df['age'],age_df['preds'],color="red")
# Add prediction line for degree =2 spline
plt.plot(age_df['age'],age_df['preds3'],color="black")

plt.ylim([40,140])
plt.show()

We specified knots at Age = 25,Age = 40, and Age =60.  From the black curve, which has a linear fit 
in each window we can clearly see the 'elbows' at those age values where the predicted curve 
changes slope while still being 'smooth' (or even continuous in a mathematical sense).



Tasks

1. Read in National Football League historical draft data located at http://myslu.stlawu.edu/~msch/data/NFLDraft.csv .  *G* is the number of games played by a player in their career. *Pick* is the position at which they were selected in the National Football League (NFL) draft. Plot G vs Pick.  The NFL allocates new players to teams by having teams select among a pool of available players.  

3. With G as the response and Pick as the predictor, fit a 4th degree polynomial, and a basis spline with four knots at 30, 60, 90 and 120 that is degree 3, and another basis spline with the same knots that is degree 1.

4. Plot the predicted functions for your models above.
    
5. How might you evaluation which method does the best job of prediction?


In [ ]:
# read in the blue jay data
nfl = pd.read_csv("http://myslu.stlawu.edu/~msch/data/NFLDraft.csv")
# remove rows with missing data
#nfl.dropna(inplace=True)

nfl.head()


In [ ]:
# the code below fills in missing values with zeros for each features/columns.
# we do this since if a player did not have a value we assume they 
# never played in a game, 'G  '
# or never started a game 'GS'
# or never had any career approximate value 'CarAV  '
nfl['G  ']=nfl['G  '].fillna(0)
nfl['GS']=nfl['GS'].fillna(0)
nfl['CarAV  ']=nfl['CarAV  '].fillna(0)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

Plot G vs. Pick

In [ ]:

plt.scatter(nfl['Pick  '],  nfl['G  '])
plt.xlabel('Pick')
plt.ylabel('Games Played ')
plt.title('Plot of Games Played vs Pick')
# Add labels and title

plt.show()

The default size for the points is large, so let's make that smaller. This plot will make the trend a bit easier to see.

In [ ]:

plt.scatter(nfl['Pick  '],  nfl['G  '],s=5)
plt.xlabel('Pick')
plt.ylabel('Games Played ')
plt.title('Plot of Games Played vs Pick')
# Add labels and title

plt.show()

We can also use an argument called alpha to make some of the points transparent.  Alpha takes
values between 0 and 1 and larger values of alpha are less transparent.

In [ ]:

plt.scatter(nfl['Pick  '],  nfl['G  '], s=6,alpha=0.3)
plt.xlabel('Pick')
plt.ylabel('Games Played ')
plt.title('Plot of Games Played vs Pick')
# Add labels and title

plt.show()

Fit a fourth degree polynomial

In [ ]:
print(nfl.info())
pick = nfl['Pick  ']
games = nfl['G  ']
# This builds the fourth degree polynomial predictors for age
# so that we have in poly_age: age, age^2, age^3, age^4
#
# pass to MS the column name in poly() and the dataframe name in fit


In [ ]:
poly_pick = MS([poly('Pick  ', degree=4)]).fit(nfl)
# Use the polynomials of age to fit a regression model an 
# ordinary least squares (OLS) one 
M = sm.OLS(games, poly_pick.transform(nfl)).fit()
# get the summary for this model
summarize(M)

In [ ]:
#make a set of 100 values from the smallest age to the largest age
pick_grid = np.linspace(pick.min(),
                       pick.max(),
                       200)
print(pick_grid)

# put those values into a dataframe called pick_df

# for the dataframe that we make below we need to name the column
# the same name as the predictor in the model so that when 
# we use this dataframe for prediction the model can find the right
# variable
pick_df = pd.DataFrame({'Pick  ': pick_grid})


In [ ]:

# make the values at which we are going to predict and transform them 
Xnew = poly_pick.transform(pick_df)
# generated predicted values at the values in age_grid
preds = M.get_prediction(Xnew)

#print(preds.summary_frame())
pick_df['preds_poly']=preds.predicted_mean

Fit a basis spline with four knots at 30, 60, 90 and 120 that is degree 3,

In [ ]:
bs_pick = MS([bs('Pick  ', degree=3,internal_knots=[30,60,90,120])])
# MS is a function that takes python sequences and makes them into a matrix
Xbs = bs_pick.fit_transform(nfl)
# The fit_transform() method in Python, particularly within the scikit-learn library, 
# serves as a combined operation of fitting and transforming data. 
# It is commonly used in data preprocessing steps for machine learning. 
# The fit() part calculates necessary parameters from the data, 
# such as mean and standard deviation in the case of scaling. 
# The transform() part then applies these calculated parameters to transform the data. 
# Using fit_transform() performs both actions sequentially in a single step, 
# which can be more efficient than calling fit() and transform() separately.

# fit a regression model to these data with y as the response and Xbs as our predictors
# call that model M2
M_nfl = sm.OLS(games, Xbs).fit()
# get the coefficient summary for M2
summarize(M_nfl)

In [ ]:
Xnew = bs_pick.transform(pick_df)
# generated predicted values at the values in age_grid
preds_spline3 = M_nfl.get_prediction(Xnew)

#print(preds.summary_frame())
pick_df['preds_spline3']=preds_spline3.predicted_mean

Fit another basis spline with the same knots that is degree 1.

In [ ]:
bs_pick2 = MS([bs('Pick  ', degree=1,internal_knots=[30,60,90,120])])
# MS is a function that takes python sequences and makes them into a matrix
Xbs2 = bs_pick2.fit_transform(nfl)
# The fit_transform() method in Python, particularly within the scikit-learn library, 
# serves as a combined operation of fitting and transforming data. 
# It is commonly used in data preprocessing steps for machine learning. 
# The fit() part calculates necessary parameters from the data, 
# such as mean and standard deviation in the case of scaling. 
# The transform() part then applies these calculated parameters to transform the data. 
# Using fit_transform() performs both actions sequentially in a single step, 
# which can be more efficient than calling fit() and transform() separately.

# fit a regression model to these data with y as the response and Xbs as our predictors
# call that model M2
M_nfl2 = sm.OLS(games, Xbs2).fit()
# get the coefficient summary for M2
summarize(M_nfl2)

In [ ]:
Xnew = bs_pick2.transform(pick_df)
# generated predicted values at the values in age_grid
preds_spline1 = M_nfl2.get_prediction(Xnew)

#print(preds.summary_frame())
pick_df['preds_spline1']=preds_spline1.predicted_mean

 Plot the predicted functions for your models above.

In [ ]:

plt.scatter(nfl['Pick  '],  nfl['G  '],s=5)
plt.xlabel('Pick')
plt.ylabel('Games Played ')
plt.title('Plot of Games Played vs Pick')


# Add prediction line to plot
plt.plot(pick_df['Pick  '],pick_df['preds_poly'] , color='yellow')
# Add prediction line to plot
plt.plot(pick_df['Pick  '],pick_df['preds_spline3'],color="purple")
# Add prediction line for degree =2 spline
plt.plot(pick_df['Pick  '],pick_df['preds_spline1'],color="blue")

plt.show()

These curves look pretty similar; let's change the scales of the axes on the graph to get a better look

In [ ]:

plt.scatter(nfl['Pick  '],  nfl['G  '],s=5)
plt.xlabel('Pick')
plt.ylabel('Games Played ')
plt.title('Plot of Games Played vs Pick')


# Add prediction line to plot
plt.plot(pick_df['Pick  '],pick_df['preds_poly'] , color='yellow')
# Add prediction line to plot
plt.plot(pick_df['Pick  '],pick_df['preds_spline3'],color="purple")
# Add prediction line for degree =2 spline
plt.plot(pick_df['Pick  '],pick_df['preds_spline1'],color="blue")

# change the range of values that will be plotted on the y-axis
plt.ylim(0,150)
plt.show()


5. How might you evaluation which method does the best job of prediction?

The best method is likely to be looking at the RMSE's for each model and evaluating their prediction.